In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import glob
import pandas as pd
from common_functions import get_parties, get_election_results, process_election

In [3]:
# These files come from http://www.infoelectoral.mir.es/infoelectoral/min/areaDescarga.html.
# Scroll down to "Extracción de datos" to obtain the zip files with the info at the
# ballot box level.
datafiles = sorted(glob.glob("/home/chema/Dropbox/data/elecciones/*.zip"))
for x in enumerate(datafiles):
    print(x)

(0, '/home/chema/Dropbox/data/elecciones/02201512_MESA.zip')
(1, '/home/chema/Dropbox/data/elecciones/02201606_MESA.zip')
(2, '/home/chema/Dropbox/data/elecciones/04201105_MESA.zip')
(3, '/home/chema/Dropbox/data/elecciones/04201505_MESA.zip')


De acuerdo a [este archivo](http://www.ine.es/daco/daco42/codmun/codmun11/11codmunmapa.htm), Madrid es la provincia 28, municipio 079. Tenemos que filtrar usando estos códigos.

In [ ]:
# Select index according to list above, then run analysis
file_idx = 2
eldata = process_election(datafiles[file_idx])
print(eldata.shape)

In [ ]:
# Need to get the percentage of vote for a given "table" (ballot box) + section
eldata_total = eldata.groupby(['section_code', 'table_code', 'dist_code'])\
    .agg({'votes': 'sum'})\
    .rename(columns = {'votes': 'total_votes'})\
    .reset_index()
eldata = pd.merge(eldata, eldata_total)
eldata['pct_votes'] = eldata['votes'] / eldata['total_votes']

In [ ]:
# Now find the boxes that had the most vote to Falange
eldata.loc[eldata['party_name'] == "LA FALANGE", :].sort_values('pct_votes', ascending = False).head(5)

In [ ]:
dist_code = '13'
section_code = '104'
table_code = 'B'
summary = eldata.loc[(eldata['section_code'] == section_code) \
                      & (eldata['dist_code'] == dist_code) \
                      & (eldata['table_code'] == table_code), :]\
    .sort_values('pct_votes', ascending = False)
display(summary)

In [ ]:
# Sanity check: do we get the same global numbers as the webpage reports
# http://www.infoelectoral.mir.es/infoelectoral/min/busquedaAvanzadaAction.html
eldata.groupby('party_name')\
    .agg({'votes': 'sum'})\
    .reset_index()\
    .sort_values('votes', ascending = False)

In [ ]:
eldata_stats = eldata.groupby('party_name')\
    .agg({'pct_votes': 'mean'})\
    .rename(columns = {'pct_votes': 'mean'})\
    .reset_index()
eldata_stats['threshold'] = 20 * eldata_stats['mean']
eldata_with_stats = pd.merge(eldata, eldata_stats)
bad_data = eldata_with_stats.loc[(eldata_with_stats['pct_votes'] > eldata_with_stats['threshold']) & (eldata_with_stats['votes'] > 20), :]\
    .groupby(['dist_code', 'section_code', 'table_code'])\
    .agg({'pct_votes': 'count'})\
    .rename(columns = {'pct_votes': 'bad_counts'})\
    .reset_index()\
    .sort_values('bad_counts', ascending = False)
display(bad_data)